# Лабораторная работа 3. Модель однофазной одноканальной замкнутой системы обслуживания
## Задание 1

In [1]:
variant <- 5
set.seed(variant)

k <- sample(c(4:9), 1)

pp1 <- runif(4)
pp2 <- runif(3)
pp3 <- runif(2)

p1 <- pp1 / sum(pp1)
p2 <- c(c(0), pp2 / sum(pp2))
p3 <- c(c(0, 0), pp3 / sum(pp3))
p4 <- c(0, 0, 0, 1)

P <- data.frame()

P <- rbind(P, p1, p2, p3, p4)

rownames(P) <- c("p1", "p2", "p3", "p4")
colnames(P) <- c("", "", "", "")

View(P)

print(paste("k =", as.character(k)))

,,,,
,<dbl>,<dbl>,<dbl>,<dbl>
p1,0.3441331,0.4604769,0.1428322,0.05255779
p2,0.0000000,0.3441697,0.2591911,0.39663917
p3,0.0000000,0.0000000,0.8964781,0.10352190
p4,0.0000000,0.0000000,0.0000000,1.00000000


[1] "k = 5"


### Граф состояний
С точностью до второго знака после запятой
![graph](./TransitionMatrix_graph--dot.png)

### Численно
Симулируем проход по матрице переходных вероятностей для трех сценариев:
1. k - 2 осмотров,
2. k - 1 осмотров,
3. k осмотров.

Выполнив эту операцию N раз для каждого сценария, получим три вектора,
содержащие N состояний. Это состояния, в которых оставалась модель после
симуляции:
$$
  Scenario_i = \{State_1 ... State_N\}, i \in \{1, 2, 3\}, \\
  State \in \{S1, S2, S3, S4\}
$$

In [2]:
# Получаем значение, соответствующее отрезку.
# v - случайная величина. Если нужно сгенерировать распределение по `p`, нужно
# подать runif(1)
# X - величины, соответствующие отрезкам.
# За отрезки отвечает параметр p, хранящий последовательность длин.
get_value_in_range <- function(v, X, p) {
    border <- 0
    for (i in seq_along(p)) {
        border <- border + p[i]
        if (v < border) {
            return(X[i])
        }
    }
}

In [3]:
make_step <- function(current_state) {
    possible_states <- P[current_state, ]

    return(get_value_in_range(runif(1), 1:4, possible_states))
}

In [4]:
walk <- function(starting_state, times) {
    if (times == 0) {
        return(starting_state)
    }

    next_state <- make_step(starting_state)

    return(walk(next_state, times - 1))
}

In [5]:
N <- 10000
test_walk <- function(first_state, k) {
    results <- c()

    for (i in 1:N) {
        results <- append(results, walk(first_state, k))
    }

    return(results)
}

In [6]:
first_state <- 1 # Выбрали произвольное.

In [7]:
Scenario1 <- test_walk(first_state, k - 2)

In [8]:
Scenario2 <- test_walk(first_state, k - 1)

In [9]:
Scenario3 <- test_walk(first_state, k)

В полученных сценариях посчитаем вероятности получения каждого состояния.

In [10]:
get_probability <- function(States, state) {
    number_of_states <- length(States[States == state])
    number_of_all_states <- length(States)

    return(number_of_states / number_of_all_states)
}

In [11]:
get_probabilities_to_stay <- function(States) {
    return(
        unlist(
            lapply(seq_along(P), function(state) get_probability(States, state))
        )
    )
}

In [12]:
Scenario1Propabilities <- get_probabilities_to_stay(Scenario1)

In [13]:
Scenario2Propabilities <- get_probabilities_to_stay(Scenario2)

In [14]:
Scenario3Propabilities <- get_probabilities_to_stay(Scenario3)

In [15]:
results <- data.frame(
    Scenario1Propabilities,
    Scenario2Propabilities,
    Scenario3Propabilities
)
colnames(results) <- c("k - 2", "k - 1", "k")
rownames(results) <- rownames(P)

results

,k - 2,k - 1,k
,<dbl>,<dbl>,<dbl>
p1,0.0414,0.0144,0.006
p2,0.1653,0.0805,0.032
p3,0.3666,0.3701,0.358
p4,0.4267,0.5350,0.604


### Теоретически

In [16]:
initial_state_probabilities <- c(1, 0, 0, 0)
transition_matrix <- data.matrix(P)
transition_matrix

,,,,
p1,0.3441331,0.4604769,0.1428322,0.05255779
p2,0.0000000,0.3441697,0.2591911,0.39663917
p3,0.0000000,0.0000000,0.8964781,0.10352190
p4,0.0000000,0.0000000,0.0000000,1.00000000


In [17]:
if (!require("matrixcalc")) {
    install.packages("matrixcalc")
}

initial_state_probabilities %*% matrix.power(transition_matrix, k - 2)

Loading required package: matrixcalc



,,,
0.04075486,0.1636169,0.3649165,0.4307117


In [18]:
initial_state_probabilities %*% matrix.power(transition_matrix, k - 1)

,,,
0.0140251,0.07507867,0.3753688,0.5355274


In [19]:
initial_state_probabilities %*% matrix.power(transition_matrix, k)

,,,
0.004826501,0.03229804,0.3579729,0.6049026


Как видно, теоретически вычисленная матрица с некоторой точностью совпадает
со значениями, полученными теоретически. При увеличении количества
экспериментов $N$ точность только увеличивается.

# Задание 2

In [20]:
variant <- 5
set.seed(variant)
k <- sample(c(10:25), 1)
# Интесивность получалась больше 1, поэтому заменил значение.
# t1 <- sample(c(14:20), 1)
t1 <- 100
t2 <- sample(c(2:5), 1)
View(data.frame(k, t1, t2))

k,t1,t2
<int>,<dbl>,<int>
11,100,4


### Численно

#### 1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал
она  же обратная вероятность того, что
программа **будет выполнена** сразу же, то есть:

In [21]:
if (!require("simmer")) {
    install.packages("simmer")
}
library(simmer)

env <- simmer("SuperDuperSim")
env

Loading required package: simmer



simmer environment: SuperDuperSim | now: 0 | next: 
{ Monitor: in memory }

In [22]:
programmers <- trajectory("programmers' path") %>%
    ## add an intake activity
    seize("server", 1) %>%
    timeout(function() rexp(1, 1 / t2)) %>%
    release("server", 1)

In [23]:
env %>%
    add_resource("server", 1) %>%
    add_generator("programmers", programmers, function() rexp(1, k / t1))

simmer environment: SuperDuperSim | now: 0 | next: 0
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programmers | monitored: 1 | n_generated: 0 }

In [24]:
env %>%
    reset() %>%
    run(1000000)

simmer environment: SuperDuperSim | now: 1e+06 | next: 1000003.64991959
{ Monitor: in memory }
{ Resource: server | monitored: TRUE | server status: 0(1) | queue status: 0(Inf) }
{ Source: programmers | monitored: 1 | n_generated: 109863 }

In [25]:
activities <- env %>% get_mon_arrivals()
activities

name,start_time,end_time,activity_time,finished,replication
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<int>
programmers0,0.6594358,0.8831157,0.2236799,TRUE,1
programmers1,4.3150260,4.6212668,0.3062408,TRUE,1
programmers2,9.9138478,17.9190320,8.0051842,TRUE,1
programmers3,17.8698740,19.4576040,1.5385720,TRUE,1
programmers4,29.1721482,35.1054429,5.9332947,TRUE,1
programmers5,30.5038747,35.2700306,0.1645877,TRUE,1
programmers6,37.6994602,51.6025574,13.9030972,TRUE,1
programmers7,44.9153141,56.7244325,5.1218751,TRUE,1
programmers8,56.5868526,63.7487193,7.0242868,TRUE,1


In [26]:
resources <- env %>% get_mon_resources()
resources

resource,time,server,queue,capacity,queue_size,system,limit,replication
<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<dbl>,<int>
server,0.6594358,1,0,1,Inf,1,Inf,1
server,0.8831157,0,0,1,Inf,0,Inf,1
server,4.3150260,1,0,1,Inf,1,Inf,1
server,4.6212668,0,0,1,Inf,0,Inf,1
server,9.9138478,1,0,1,Inf,1,Inf,1
server,17.8698740,1,1,1,Inf,2,Inf,1
server,17.9190320,1,0,1,Inf,1,Inf,1
server,19.4576040,0,0,1,Inf,0,Inf,1
server,29.1721482,1,0,1,Inf,1,Inf,1


#### 1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал
она  же обратная вероятность того, что
программа **будет выполнена** сразу же, то есть:

In [27]:
EPS <- 0.0001 # Должно быть 0, но в модели присутствуют некоторые погрешности.
queue <- resources$queue
income_count <- length(activities$name)
programs_starts <- length(
    subset(activities, (activities$end_time - activities$start_time - activities$activity_time) > EPS)$name
)

programs_starts
income_count

[1] 48168

[1] 109862

In [28]:
program_wont_be_executed_immediately <- programs_starts / income_count
program_wont_be_executed_immediately

[1] 0.438441

#### 2. Среднее время до получения пользователем результатов реализации.

In [38]:
finished_activity_time <- mean(activities$end_time - activities$start_time)
finished_activity_time

[1] 7.144037

#### 3. Среднее количество программ, ожидающих выполнения на сервере.

In [30]:
mean_queue <- program_wont_be_executed_immediately^2 / (1 - program_wont_be_executed_immediately)
mean_queue

[1] 0.3423157

### Теоретически

#### 1. Вероятность того, что программа не будет выполнена сразу же, как только она поступила на терминал
она  же обратная вероятность того, что
программа **будет выполнена** сразу же, то есть:
$$
1 - P_0 = 1 - (1 - \rho) = \rho = \frac{\lambda}{\mu}
$$
где $\lambda$ - интенсивность, с которой заявки приходят:
$$
\lambda = \frac{k}{t_1} \\
$$

In [31]:
income_intensity <- k / t1
income_intensity

[1] 0.11

а $\mu$ - интенсивность обслуживания:
$$
\mu = \frac{1}{t_2}
$$

In [32]:
process_intensity <- 1 / t2
process_intensity

[1] 0.25

In [33]:
program_wont_be_executed_immediately <- income_intensity / process_intensity
program_wont_be_executed_immediately

[1] 0.44

#### 2. Среднее время до получения пользователем результатов реализации.

Оно же среднее время пребывания заявки в системе по формуле Литтла:
$$
T_{\text{сист}} = \frac{1}{\mu(1 - \rho)}
$$

In [34]:
time_to_get <- 1 / process_intensity / (1 - program_wont_be_executed_immediately)
time_to_get

[1] 7.142857

#### 3. Среднее количество программ, ожидающих выполнения на сервере.

Она же средняя длина очереди $L_{\text{оч}} = \frac{\rho ^ 2}{1 - \rho}$:

In [35]:
mean_queue <- program_wont_be_executed_immediately^2 / (1 - program_wont_be_executed_immediately)
mean_queue

[1] 0.3457143

Как видно, все значения сошлись.